In [2]:
import numpy as np
import tensorflow as tf
from scipy.io import loadmat
from sklearn.decomposition import IncrementalPCA
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Load data
data = loadmat("umist_cropped.mat")['facedat'][0]
face, target = [], []
label = 0
for batch in data:
    for sample in batch.T:
        sample = sample.T.reshape(-1)
        face.append(sample)
        target.append(label)
    label += 1
face, target = np.array(face), np.array(target)

In [3]:
# Encode target labels
label_encoder = LabelEncoder()
encoded_target = label_encoder.fit_transform(target)

In [4]:
# Feature reduction with Incremental PCA
pca = IncrementalPCA(n_components=260)
face_reduced = pca.fit_transform(face)

In [5]:
# Clustering with Gaussian Mixture Model
gmm = GaussianMixture(n_components=20, covariance_type='spherical').fit(face_reduced)
new_face, new_target = gmm.sample(1000)
new_target = tf.keras.utils.to_categorical(new_target, num_classes=label)


In [6]:
# Stratified Splitting of the dataset
x_train, x_temp, y_train, y_temp = train_test_split(new_face, new_target, test_size=0.4, stratify=new_target, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [7]:
# Model Definition
def create_deep_and_cross_model(input_shape, num_classes, hidden_units=[64, 64]):
    input_layer = tf.keras.Input(shape=input_shape)

    # Cross Network
    cross = input_layer
    for _ in hidden_units:
        units = input_layer.shape[-1]
        x = tf.keras.layers.Dense(units, kernel_regularizer='l2')(input_layer)
        cross = input_layer * x + cross
    cross = tf.keras.layers.BatchNormalization()(cross)

    # Deep Network
    deep = input_layer
    for units in hidden_units:
        deep = tf.keras.layers.Dense(units, kernel_regularizer='l2')(deep)
        deep = tf.keras.layers.BatchNormalization()(deep)
        deep = tf.keras.layers.ReLU()(deep)

    # Combine Networks
    merged = tf.keras.layers.concatenate([cross, deep])
    output_layer = tf.keras.layers.Dense(units=num_classes, activation='softmax')(merged)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
    return model


In [8]:
# Create and Train the Model
model = create_deep_and_cross_model(input_shape=x_train.shape[1:], num_classes=label)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, callbacks=[early_stopping])


Epoch 1/100
19/19 [==============================] - 1s 10ms/step - loss: 10.1726 - accuracy: 0.0817 - val_loss: 11.9153 - val_accuracy: 0.1050
Epoch 2/100
19/19 [==============================] - 0s 3ms/step - loss: 8.6663 - accuracy: 0.3300 - val_loss: 9.8992 - val_accuracy: 0.1700
Epoch 3/100
19/19 [==============================] - 0s 3ms/step - loss: 7.6898 - accuracy: 0.5950 - val_loss: 8.8458 - val_accuracy: 0.2300
Epoch 4/100
19/19 [==============================] - 0s 3ms/step - loss: 6.9631 - accuracy: 0.8100 - val_loss: 8.1117 - val_accuracy: 0.3400
Epoch 5/100
19/19 [==============================] - 0s 3ms/step - loss: 6.3774 - accuracy: 0.9117 - val_loss: 7.5438 - val_accuracy: 0.4200
Epoch 6/100
19/19 [==============================] - 0s 3ms/step - loss: 5.9209 - accuracy: 0.9717 - val_loss: 7.0771 - val_accuracy: 0.4950
Epoch 7/100
19/19 [==============================] - 0s 3ms/step - loss: 5.5400 - accuracy: 0.9833 - val_loss: 6.6790 - val_accuracy: 0.5750
Epoch 8/10

In [9]:
# Evaluate the Model
predict = model.predict(x_test)
predict_labels, true_labels = [np.argmax(one_hot) for one_hot in predict], [np.argmax(one_hot) for one_hot in y_test]
print(classification_report(true_labels, predict_labels))

7/7 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        23
           1       0.95      1.00      0.97        18
           2       1.00      1.00      1.00         8
           3       0.67      0.67      0.67         3
           4       1.00      1.00      1.00         7
           5       0.94      1.00      0.97        16
           6       1.00      1.00      1.00        12
           7       0.91      0.91      0.91        11
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00        12
          11       1.00      1.00      1.00         1
          12       1.00      0.71      0.83         7
          13       0.88      1.00      0.93         7
          14       1.00      1.00      1.00        10
          15       0.60      0.38      0.46         8
          16       0.92      1